# Trans-World Oil:  Linear Optimization
### Sean Olson

In [1]:
# import libraries
from optlang import Model, Variable, Constraint, Objective
import pandas as pd

In [23]:
# import data
tankerleasecost = 8600 # lease cost per day
usdistillatecost = 19.80

demand = pd.read_excel('trans_world_oil_company_data.xlsx', sheet_name = 'Demand')
demand['Gasoline'] = demand['Gasoline'] * 1000
demand['Distillate'] = demand['Distillate'] * 1000
display(demand)

refinerycost = pd.read_excel('trans_world_oil_company_data.xlsx', sheet_name = 'RefineryCosts')
refinerycost['Total'] = refinerycost['Gasoline Yield per Crude BBL'] + refinerycost['Distillate Yield per Crude BBL']
display(refinerycost)

variablecost = pd.read_excel('trans_world_oil_company_data.xlsx', sheet_name = 'VariableCosts')
display(variablecost)

tankerusage = pd.read_excel('trans_world_oil_company_data.xlsx', sheet_name = 'TankerUsageFactors')
display(tankerusage)

supply = pd.read_excel('trans_world_oil_company_data.xlsx', sheet_name = 'Supply')
display(supply)

,Area,Gasoline,Distillate
0,Australia,9000.0,21000.0
1,Japan,3000.0,12000.0
2,Philippines,5000.0,8000.0
3,New Zealand,5400.0,8700.0
4,Total,22400.0,49700.0


,"Location, Crude, PI",Country,Cost of Crude,Inc. Ship. Cost,Inc. Ref. Cost,Total Costs,Gasoline Yield per Crude BBL,Distillate Yield per Crude BBL,Total
0,"Brunei Crude, Low (BLA)",Australia,20.5,0.78,0.36,21.64,0.259,0.688,0.947
1,"Brunei Crude, High (BHA)",Australia,20.5,0.78,0.84,22.12,0.365,0.573,0.938
2,"Iran Crude, Low (ILA)",Australia,18.5,1.86,0.45,20.81,0.186,0.732,0.918
3,"Iran Crude, High (IHA)",Australia,18.5,1.86,0.90,21.26,0.312,0.608,0.920
4,"Brunei Crude, Low (BLJ)",Japan,20.5,0.72,0.48,21.70,0.259,0.688,0.947
5,"Brunei Crude, High (BHJ)",Japan,20.5,0.72,1.02,22.24,0.350,0.588,0.938
6,"Iran Crude, Low (ILJ)",Japan,18.5,1.77,0.60,20.87,0.186,0.732,0.918
7,"Iran Crude, High (IHJ)",Japan,18.5,1.77,1.17,21.44,0.300,0.620,0.920


,From / To,New Zealand,Philippines
0,Australia,0.3,0.45
1,Japan,0.3,0.60
2,US,2.1,1.65


,Between,Australia,Japan,US
0,Iran,0.12,0.110,NaN
1,Borneo,0.05,0.045,NaN
2,Philippines,0.02,0.010,0.15
3,New Zealand,0.01,0.060,0.18


,Supply,BBL/Day,Type
0,Brunei (Borneo),40000,Fixed
1,Iran,60000,Available
2,US Distillate,12000,Available


### Define Objective Function
Now we must identify the objective function for the case.  Given that all the data for the case is cost data then it is safe to assume that the objective function needs to be **Minimize Costs**.

Now we have to define our objective function:

*Total Costs = Cost of Refined Products + Shipment to Other Markets + Tanker Lease*

*Cost of Refined Products = AustraliaCost(Gas + Distillate) + JapanCost(Gas + Distillate)*

### Define Constraints
##### Crude Oil Supply:
- Brunei <= 40000
- Iran <= 60000

##### Distillate Supply:
- US <= 12000

##### Refinery Constraint
- Australia <= 50000
- Japan <= 30000

##### Market Demand:  Gas
- Australia >= 9000
- Japan >= 3000
- Philippines >= 5000
- New Zealand >= 5400

##### Market Demand:  Distillate
- Australia >= 21000
- Japan >= 12000
- Philippines >= 8000
- New Zealand >= 8700

##### Tanker Capacity
- Tanker Capacity <= 6900
  
##### Gas Shipped from Refineries to Other Markets
- Australia -> Philippines >= 0.45
- Australia -> New Zealand >= 0.30
- Japan -> Philippines >= 0.60
- Japan -> New Zealand >= 0.30

##### Distillate Shipped from Refineries to Other Markets
- US -> Philippines >= 1.65
- US -> New Zealand >= 2.10
- Australia -> Philippines >= 0.45
- Australia -> New Zealand >= 0.30
- Japan -> Philippines >= 0.60
- Japan -> New Zealand >= 0.30

In [22]:
# define objective function
# decision variables


# refinery constraints
c1 = aus_brl+aus_brh+aus_irl+aus_irh <= 50000

c2 = jpn_brl+jpn_brh+jpn_irl+jpn_irh <= 30000

c3 = aus_brl+aus_brh+jpn_brl+jpn_brh <= 40000
c4 = aus_irl+aus_irh+jpn_irl+jpn_irh <= 60000

# gas production constraints
c5 = aus_gas >= 9000
c6 = (aus_brl*0.259)+(aus_brh*0.365)+(aus_irl*0.186)+(aus_irh*0.312) - aus_gas - aus_nz_gas >= 5000
c7 = (aus_brl*0.259)+(aus_brh*0.365)+(aus_irl*0.186)+(aus_irh*0.312) - aus_gas - aus_phl_gas >= 5400

c8 = jpn_gas >= 3000
c9 = (jpn_brl*0.259)+(jpn_brh*0.350)+(jpn_irl*0.186)+(jpn_irh*0.300) - jpn_gas - jpn_nz_gas >= 5000
c10 = (jpn_brl*0.259)+(jpn_brh*0.350)+(jpn_irl*0.186)+(jpn_irh*0.300) - jpn_gas - jpn_phl_gas >= 5400

# distillate production constraints
c11 = us_dis <= 12000

c12 = aus_dis >= 21000
c13 = (aus_brl*0.688)+(aus_brh*0.573)+(aus_irl*0.732)+(aus_irh*0.608) + us_phl_dis - aus_dis - aus_nz_dis >= 8000
c14 = (aus_brl*0.688)+(aus_brh*0.573)+(aus_irl*0.732)+(aus_irh*0.608) + us_nz_dis - aus_dis - aus_phl_dis >= 8700

c15 = jpn_dis >= 12000
c16 = (jpn_brl*0.688)+(jpn_brh*0.588)+(jpn_irl*0.732)+(jpn_irh*0.620) + us_phl_dis - jpn_dis - jpn_nz_dis >= 8000
c17 = (jpn_brl*0.688)+(jpn_brh*0.588)+(jpn_irl*0.732)+(jpn_irh*0.620) + us_nz_dis - jpn_dis - jpn_phl_dis >= 8700

# tanker capacity
c18 = ((aus_irl+aus_irh)*0.12)+((aus_brl+aus_brh)*0.05)+((jpn_irl+jpn_irh)*0.11)+((jpn_brl+jpn_brh)*0.045)\
+((aus_phl_gas+aus_phl_dis)*0.02)+((aus_nz_gas+aus_nz_dis)*0.01)+((jpn_phl_gas+jpn_phl_dis)*0.01)\
+((jpn_nz_gas+jpn_nz_dis)*0.06)+(us_phl_dis*0.15)+(us_nz_dis*0.18)-(1000*tkrs) <= 6900





# objective function
# refining cost
aus_ref_cost = (aus_brl*21.64)+(aus_brh*22.12)+(aus_irl*20.81)+(aus_irh*21.26)
jpn_ref_cost = (jpn_brl*21.70)+(jpn_brh*22.24)+(jpn_irl*20.87)+(jpn_irh*21.44)

# us distillate cost
usdist_cost = (phl_dis+nz_dis)*19.80

# total cost to ship to other markets
aus_phl_cost = ((aus_brl*0.947)+(aus_brh*0.938)+(aus_irl*0.918)+(aus_irh*0.920))*0.45
aus_nz_cost = ((aus_brl*0.947)+(aus_brh*0.938)+(aus_irl*0.918)+(aus_irh*0.920))*0.30

jpn_phl_cost = ((jpn_brl*0.947)+(jpn_brh*0.938)+(jpn_irl*0.918)+(jpn_irh*0.920))*0.60
jpn_nz_cost = ((jpn_brl*0.947)+(jpn_brh*0.938)+(jpn_irl*0.918)+(jpn_irh*0.920))*0.30

us_phl_cost = us_dis*1.65
us_nz_cost = us_dis*2.1

# tanker cost
tkrs_cost = tkrs*8600




obj = Objective(, direction='min')

# run the model
model = Model(name='Trans-world Oil')
model.add([])
model.objective = obj
status = model.optimize()
print("status:", model.status)
print("objective value:", model.objective.value)
print("----------")
for var_name, var in model.variables.items():
    print(var_name, "=", var.primal)

ContainerAlreadyContains: Container '<optlang.container.Container object at 0x11f3025c0>' already contains an object with name 'brx'.

In [ ]:
crude_cost = (x1*20.5) + (x2*18.5)
ship_cost = (((x1*0.75)/6900)*11*8600)+(((x2*1.82)/6900)*35*8600)
aus_refinery_cost = (((x12+x13+x14)*0.36)/((0.259*x20)+(0.688*x21))) + (((x15+x13+x14)*0.84)/((0.365*x20)+(0.573*x21))) + (((x12+x15+x14)*0.45)/((0.186*x20)+(0.732*x21))) + (((x12+x15+x13)*0.90)/((0.312*x20)+(0.608*x21)))
jpn_refinery_cost = (((x19+x17+x18)*1.02)/((0.350*x22)+(0.588*x23))) + (((x16+x19+x18)*0.60)/((0.186*x22)+(0.732*x23))) + (((x16+x19+x17)*1.17)/((0.300*x22)+(0.620*x23))) + (((x16+x17+x18)*0.48)/((0.259*x22)+(0.688*x23)))
ausphlcost = (((0.45*(x20-x5-x7) + (0.45*(x21-x6-x8)))/6900)*7*8600)
ausnzcost = ((((0.30*(x20-x3-x7)) + (0.30*(x21-x4-x8)))/6900)*2*8600)
jpnphlcost = ((((0.60*(x22-x5-x9)) + (0.60*(x23-x6-x10)))/6900)*14*8600)
jpnnzcost = ((((0.30*(x22-x3-x9)) + (0.30*(x23-x4-x10)))/6900)*18*8600)
usphlcost = ((((1.65*(x11-x6)))/6900)*30*8600)
usnzcost = ((((2.10*(x11-x4)))/6900)*30*8600)